In [1]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
# from keras.layers.embeddings import Embedding
## Plotly
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

Using TensorFlow backend.


In [2]:
filepath = "../sample.csv"
df = pd.read_csv(filepath, sep = ';', error_bad_lines=False)
df = df.dropna()
df = df[df.label.apply(lambda x: x !="")]
df = df[df.text.apply(lambda x: x !="")]

b'Skipping line 239: expected 2 fields, saw 4\nSkipping line 699: expected 2 fields, saw 3\nSkipping line 1154: expected 2 fields, saw 3\nSkipping line 1216: expected 2 fields, saw 3\nSkipping line 1238: expected 2 fields, saw 3\nSkipping line 1240: expected 2 fields, saw 3\nSkipping line 1954: expected 2 fields, saw 3\nSkipping line 1956: expected 2 fields, saw 3\nSkipping line 1960: expected 2 fields, saw 3\nSkipping line 1964: expected 2 fields, saw 3\nSkipping line 2951: expected 2 fields, saw 3\nSkipping line 2968: expected 2 fields, saw 3\nSkipping line 2972: expected 2 fields, saw 3\nSkipping line 2981: expected 2 fields, saw 3\nSkipping line 3464: expected 2 fields, saw 3\nSkipping line 3919: expected 2 fields, saw 3\nSkipping line 4908: expected 2 fields, saw 3\n'


In [3]:
df.count()

label    4963
text     4963
dtype: int64

In [4]:
import re
from nltk.stem.snowball import SnowballStemmer
### Text Normalizing function. Part of the following function was taken from this link. 
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text
# apply the above function to df['text']
df['text'] = df['text'].map(lambda x: clean_text(x))

In [5]:
df.head()

,label,text
0,1,awar 180 day problem differ
1,1,report use dashboard come empti
2,1,sourc report show chart data select custom run...
3,1,puzzl
4,1,report name open issu account


In [6]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=60, padding = "post")

In [7]:
data

array([[ 879, 1360,   43, ...,    0,    0,    0],
       [  78,   32,  880, ...,    0,    0,    0],
       [ 881,   78,   61, ...,    0,    0,    0],
       ...,
       [ 390, 2246,    0, ...,    0,    0,    0],
       [ 466,  158,  184, ...,    0,    0,    0],
       [ 158, 2247, 2248, ...,    0,    0,    0]], dtype=int32)

In [8]:
len(data)

4963

In [9]:
l = df.label[0:5]
l

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [11]:
import torch
batch_size = 5
batch_count = int(len(data)/batch_size)-1
train_iterator = []
# whole_labels = 
for i in range(batch_count) :
    start_index = (i*batch_size)
    end_index = start_index + batch_size
    labels = torch.from_numpy(np.array(df.label[start_index:end_index], dtype=np.float32))

    text_data = np.transpose(data[start_index:end_index])
    text_data = torch.from_numpy(np.array(text_data, dtype=np.int64)) #np.int64 - long type, np.int32 - int type
    val = {'label':labels, 'text':text_data}
    train_iterator.append(val)
    

In [12]:
t = data[0:5]
t

array([[ 879, 1360,   43,   64,  141,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [  78,   32,  880,  194,  599,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [ 881,   78,   61, 1361,  112,  170,   14,  341,  112,  226,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,   

In [13]:
# print(tokenizer.word_counts)
print(tokenizer.document_count)
# print(tokenizer.word_index)
# print(tokenizer.word_d?ocs)


4963


In [14]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [15]:
# INPUT_DIM = len(TEXT.vocab)
INPUT_DIM = len(tokenizer.word_index)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [16]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [17]:
criterion = nn.BCEWithLogitsLoss()

In [18]:
device = "cpu"
model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch['text']).squeeze(1)
        
        loss = criterion(predictions, batch['label'])
        
        acc = binary_accuracy(predictions, batch['label'])
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        print("\rLoss : {} Acc : {}".format(epoch_loss, epoch_acc), end="")
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch['text']).squeeze(1)
            
            loss = criterion(predictions, batch['label'])
            
            acc = binary_accuracy(predictions, batch['label'])

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
#     valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#     print(f' | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

Loss : 665.1725817322731 Acc : 615.4000106155872| Epoch: 01 | Train Loss: 0.671 | Train Acc: 62.10%
Loss : 635.9929367154837 Acc : 637.800011485815| Epoch: 02 | Train Loss: 0.642 | Train Acc: 64.36%
Loss : 666.6392587721348 Acc : 615.8000103533268| Epoch: 03 | Train Loss: 0.673 | Train Acc: 62.14%
Loss : 675.6586938649416 Acc : 612.6000097692013| Epoch: 04 | Train Loss: 0.682 | Train Acc: 61.82%
Loss : 675.91584636271 Acc : 614.200009778142| Epoch: 05 | Train Loss: 0.682 | Train Acc: 61.98%
Loss : 674.816470220685 Acc : 615.6000097692013| Epoch: 06 | Train Loss: 0.681 | Train Acc: 62.12%
Loss : 674.6375990062952 Acc : 616.200009778142| Epoch: 07 | Train Loss: 0.681 | Train Acc: 62.18%
Loss : 672.7166714370251 Acc : 611.200009778142| Epoch: 08 | Train Loss: 0.679 | Train Acc: 61.68%
Loss : 659.3773667067289 Acc : 614.8000101149082| Epoch: 09 | Train Loss: 0.665 | Train Acc: 62.04%
Loss : 660.5386760830879 Acc : 614.6000099927187| Epoch: 10 | Train Loss: 0.667 | Train Acc: 62.02%
